<a href="https://colab.research.google.com/github/regulate-tech/nhstech/blob/main/subject-paper/nhstech_project_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Obtain ERIC data on cost of storing medical records

This notebook creates a combined dataset that we can use to analyse spending on storing medical records in the NHS. The notebook:

- Collects all relevant releases from the [ERIC dataset](https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection) on medical records
- Tidies the data, adds a utility column for total spending, de-duplicates trusts with the same code
- Checks the data, and joins it into a single file.

For the data definitions, see e.g. [the 'ERIC 2023/24 - Data Definitions' file here](https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection/management-information---provisional-summary-figures-for-2023-24).

In [1]:
import os
import re
from urllib.parse import unquote, urlparse

import chardet
import numpy as np
import pandas as pd
import requests

First, fetch all the CSV files from the [published ERIC collection](https://digital.nhs.uk/data-and-information/publications/statistical/estates-returns-information-collection) and store them in a local directory. As there are relatively few of these, we just store the links manually.

In [2]:
DATA_URLS = [
    'https://files.digital.nhs.uk/AF/F737A9/ERIC%20-%202023_24%20-%20Trust%20data.csv',
    'https://files.digital.nhs.uk/C7/C7C191/ERIC%20-%202023_24%20-%20Site%20data.csv',
    'https://files.digital.nhs.uk/FF/4DC8D8/ERIC%20-%202023_24%20-%20PFI%20data.csv',
    'https://files.digital.nhs.uk/FB/BE3AC8/ERIC%20-%20202223%20-%20Trust%20data.csv',
    'https://files.digital.nhs.uk/41/5787C9/ERIC%20-%202022_23%20-%20Site%20data.csv',
    'https://files.digital.nhs.uk/42/D5A005/ERIC%20-%202022_23%20-%20PFI%20data.csv',
    'https://files.digital.nhs.uk/08/84C46C/ERIC%20-%20202122%20-%20Trust%20data.csv',
    'https://files.digital.nhs.uk/EE/7E330D/ERIC%20-%20202122%20-%20Site%20Data%20v3.csv',
    'https://files.digital.nhs.uk/D3/D0DFD3/ERIC%20-%20202122%20-%20PFI%20data%20-%20v2.csv',
    'https://files.digital.nhs.uk/81/4A77B0/ERIC%20-%20202021%20-%20Trust%20data.csv',
    'https://files.digital.nhs.uk/0F/46F719/ERIC%20-%20202021%20-%20Site%20data%20v2.csv',
    'https://files.digital.nhs.uk/5F/4B00BC/ERIC%20-%20202021%20-%20PFI%20data.csv',
    'https://files.digital.nhs.uk/84/07227E/ERIC%20-%20201920%20-%20TrustData.csv',
    'https://files.digital.nhs.uk/11/BC1043/ERIC%20-%20201920%20-%20SiteData%20-%20v2.csv',
    'https://files.digital.nhs.uk/51/8C7C23/ERIC%20-%20201920%20-%20PFIData.csv',
    'https://files.digital.nhs.uk/83/4AF81B/ERIC%20-%20201819%20-%20TrustData%20v4.csv',
    'https://files.digital.nhs.uk/63/ADBFFF/ERIC%20-%20201819%20-%20SiteData%20v4.csv',
    'https://files.digital.nhs.uk/F6/791B8F/ERIC%20-%20201819%20-%20PFIData%20v3.csv',
    'https://files.digital.nhs.uk/1B/7C75CF/ERIC-201718-TrustData.csv',
    'https://files.digital.nhs.uk/A8/188D99/ERIC-201718-SiteData.csv',
    'https://files.digital.nhs.uk/09/928620/ERIC-201718-PFIData.csv'
]

# Fetch raw data 

Fetch files, formatting filenames consistently.

In [3]:
DATA_DIR = "./data/source/eric/"
for url in DATA_URLS:
    try:
        response = requests.get(url)
        response.raise_for_status()
        # Format filenames if necessary.
        filename = unquote(os.path.basename(urlparse(url).path))
        filename = filename.replace(" ", "-")
        if "_" in filename:
            parts = filename.split("_")
            filename = parts[0] + parts[1]
        with open(os.path.join(DATA_DIR, filename), 'wb') as f:
            f.write(response.content)
    except Exception as e:
        print(f"Failed to download {url}: {e}")

Make everything UTF-8.

In [4]:
csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
for filename in csv_files:
  filepath = os.path.join(DATA_DIR, filename)
  with open(filepath, 'rb') as f:
    rawdata = f.read()
  result = chardet.detect(rawdata)
  encoding = result["encoding"]
  if encoding != 'utf-8':
    try:
      with open(filepath, 'r', encoding=encoding) as f:
        data = f.read()
      with open(filepath, 'w', encoding='utf-8') as f:
        f.write(data)
    except Exception as e:
      print(f"Error converting {filename}: {e}")

# Extract the relevant columns

The ERIC data has lots of data points in it, but we only care about the data on the use of medical records, so extract those columns. These are the columns starting `mrc_*` (short for 'medical records'). 

In [13]:
def process_csv_files():
  """
  Get the trust files for all years, and store any columns containing 'Medical Records'
  into a single large dict, keyed off trust name and code.
  """
  trust_data = {}

  # We're only interested in trust records for the moment.
  csv_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv') and "Trust" in f]
  for filename in csv_files:
    filepath = os.path.join(DATA_DIR, filename)

    year_code = re.search(r'(\d{6})', filename)
    if year_code:
      year_code = year_code.group(1)
    else:
      year_code = "Unknown" 

    try:
      # This file has two superfluous header rows, which we need to skip.
      if "202324---Trust-data---Provisional" in filepath:
        df = pd.read_csv(filepath, skiprows=2)
      else:
        df = pd.read_csv(filepath)
      df.head()

      if "Trust Code" in df.columns and "Trust Name" in df.columns:
        for index, row in df.iterrows():
          trust_code = row["Trust Code"]
          trust_name = row["Trust Name"]
          if (trust_code, trust_name) not in trust_data:
            trust_data[(trust_code, trust_name)] = {}
              
          for col in df.columns:
            if "Medical Records" in col:
              # Store the data with the column name plus year code
              trust_data[(trust_code, trust_name)][col + "_" + year_code] = row[col]
    except Exception as e:
      print(f"Error processing file {filename}: {e}")

  return trust_data

trust_data_dict = process_csv_files()

In [16]:
# Now transform the dict into a dataframe.
trust_df = pd.DataFrame.from_dict(trust_data_dict, orient='index')
trust_df = trust_df.reset_index()

# And clean up the column names.
trust_df = trust_df.rename(columns={'level_0': 'trust_code', 'level_1': 'trust_name'})
replacements = {
    r'Medical Records cost - Onsite \(£\)_(\d{6})': r'mrc_on_\1',
    r'Medical Records cost - Offsite \(£\)_(\d{6})': r'mrc_off_\1',
    r'Medical Records volume - Onsite \(records\)_(\d{6})': r'mrv_on_\1',
    r'Medical Records volume - Offsite \(records\)_(\d{6})': r'mrv_off_\1',
    # TODO - investigate what these columns are and whether they are useful, and 
    # handle them in the rest of our processing.
    r'Type of Medical Records \(Select\)_(\d{6})': r'mrc_type_\1',
    r'Medical Records service provision \(Select\)_(\d{6})': r'mrc_service_provision_\1'
}
for old, new in replacements.items():
    trust_df.columns = trust_df.columns.str.replace(old, new, regex=True)
trust_df.head(2)

,trust_code,trust_name,mrc_on_201819,mrc_off_201819,mrc_type_201819,mrc_service_provision_201819,mrc_on_202324,mrc_off_202324,mrc_type_202324,mrc_service_provision_202324,...,mrc_type_202122,mrc_service_provision_202122,mrc_on_202223,mrc_off_202223,mrc_type_202223,mrc_service_provision_202223,mrc_on_201718,mrc_off_201718,mrc_type_201718,mrc_service_provision_201718
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,"2,871,339","145,631",3. Mixed,Internal,"5,557,232","524,104",3. Mixed,Internal,...,3. Mixed,Internal,"5,629,069","296,103",3. Mixed,Internal,"3,612,846","203,096",Mixed,Internal
1,R1A,WORCESTERSHIRE HEALTH AND CARE NHS TRUST,"169,347","118,345",3. Mixed,Hybrid,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"155,193","107,682",Mixed,Hybrid


## Tidy up the data

Now, create a nice clean table with 'Trust Code' and 'Trust Name' and each column in ascending year order: and tidy up the number formatting.

In [17]:
# Order columns by year.
trust_df = trust_df.reindex(sorted(trust_df.columns, key=lambda x: x.split()[-1] if '_' in x else x), axis=1)

# And put the name and code columns first. 
mrc_columns = [col for col in trust_df.columns if col.startswith("mrc_")]
mrc_columns.sort()
cols = ['trust_code', 'trust_name'] + mrc_columns
trust_df = trust_df[cols]

# Drop the null column (this is an artefact of how we process the data).
trust_df = trust_df[~trust_df.trust_name.isnull()]
trust_df[trust_df.trust_code == 'R0A']

,trust_code,trust_name,mrc_off_201718,mrc_off_201819,mrc_off_201920,mrc_off_202021,mrc_off_202122,mrc_off_202223,mrc_off_202324,mrc_on_201718,...,mrc_service_provision_202122,mrc_service_provision_202223,mrc_service_provision_202324,mrc_type_201718,mrc_type_201819,mrc_type_201920,mrc_type_202021,mrc_type_202122,mrc_type_202223,mrc_type_202324
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,"203,096","145,631","277,200","458,075","438,847","296,103","524,104","3,612,846",...,Internal,Internal,Internal,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed


In [18]:
# Replace null values, and tidy up number formatting.
number_columns = [col for col in trust_df.columns if col.startswith("mrc_on") or col.startswith("mrc_off")]
trust_df[number_columns] = trust_df[number_columns].apply(lambda x: x.str.replace(',', ''))
trust_df[number_columns] = trust_df[number_columns].apply(pd.to_numeric, errors='coerce')
trust_df.fillna({col: 0 for col in number_columns}, inplace=True)
trust_df[trust_df.trust_code == 'R0A']

,trust_code,trust_name,mrc_off_201718,mrc_off_201819,mrc_off_201920,mrc_off_202021,mrc_off_202122,mrc_off_202223,mrc_off_202324,mrc_on_201718,...,mrc_service_provision_202122,mrc_service_provision_202223,mrc_service_provision_202324,mrc_type_201718,mrc_type_201819,mrc_type_201920,mrc_type_202021,mrc_type_202122,mrc_type_202223,mrc_type_202324
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,203096.0,145631.0,277200.0,458075.0,438847.0,296103.0,524104.0,3612846.0,...,Internal,Internal,Internal,Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed,3. Mixed


In [20]:
len(trust_df)

279

## De-duplicate trust entries

Some of the entries in this file have different names but the same codes, typically where they have changed name. If we de-dupe these, and glue the names together, we end up with records for 234 trusts rather than 279.

In [29]:
# Confirm that there are aren't any currently active duplicate codes - all the dupes
# are trusts that were active in the past.
currently_active_trusts = trust_df[trust_df['mrc_on_202324'].notna() | trust_df['mrc_off_202324'].notna()]
duplicates = currently_active_trusts[currently_active_trusts['trust_code'].duplicated(keep=False)]
# An example of a duplicate entry.
trust_df[trust_df.trust_code == 'RYR']

,trust_code,trust_name,mrc_off_201718,mrc_off_201819,mrc_off_201920,mrc_off_202021,mrc_off_202122,mrc_off_202223,mrc_off_202324,mrc_on_201718,...,mrc_service_provision_202122,mrc_service_provision_202223,mrc_service_provision_202324,mrc_type_201718,mrc_type_201819,mrc_type_201920,mrc_type_202021,mrc_type_202122,mrc_type_202223,mrc_type_202324
218,RYR,WESTERN SUSSEX HOSPITALS NHS FOUNDATION TRUST,30000.0,58000.0,53734.0,12986.0,0.0,0.0,0.0,2708018.0,...,NaN,NaN,NaN,Mixed,3. Mixed,3. Mixed,3. Mixed,NaN,NaN,NaN
264,RYR,UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST,0.0,0.0,0.0,0.0,3068085.0,2721185.0,3200340.0,0.0,...,Hybrid,Internal,Hybrid,NaN,NaN,NaN,NaN,3. Mixed,3. Mixed,3. Mixed


In [32]:
# Group on trust code: sum numeric values, concatenate string values (like trust name).
def join_strings(x):
    return ' / '.join(x.unique())

numeric_cols = trust_df.select_dtypes(include=['number']).columns.tolist()
string_cols = trust_df.select_dtypes(include=['string', 'object']).columns.tolist()
string_cols.remove("trust_code")
agg_dict = {
    **{col: 'sum' for col in numeric_cols},
    **{col: lambda x: ', '.join(x.dropna()) for col in string_cols}
}
df = trust_df.groupby('trust_code').agg(agg_dict)
df.reset_index(inplace=True)

string_cols.remove("trust_name") # We don't want this twice.
df = df[['trust_code', 'trust_name'] + numeric_cols + string_cols] # Order the columns nicely.
df.head()
print(len(trust_df), 'records in the raw dataset:', len(df), 'records once deduplicated on trust code.')

279 records in the raw dataset: 234 records once deduplicated on trust code.


## Calculate total spending

It's useful to have a column on total spending - this is onsite and offsite medical records spending combined.

In [33]:
mrc_on_columns = [col for col in df.columns if col.startswith('mrc_on_')]
year_codes = [col[-6:] for col in mrc_on_columns]
for year_code in year_codes:
  df['mrc_total_' + year_code] = df['mrc_on_' + year_code] + df['mrc_off_' + year_code]
df[df.trust_code=='R0A']

,trust_code,trust_name,mrc_off_201718,mrc_off_201819,mrc_off_201920,mrc_off_202021,mrc_off_202122,mrc_off_202223,mrc_off_202324,mrc_on_201718,...,mrc_type_202122,mrc_type_202223,mrc_type_202324,mrc_total_201718,mrc_total_201819,mrc_total_201920,mrc_total_202021,mrc_total_202122,mrc_total_202223,mrc_total_202324
0,R0A,MANCHESTER UNIVERSITY NHS FOUNDATION TRUST,203096.0,145631.0,277200.0,458075.0,438847.0,296103.0,524104.0,3612846.0,...,3. Mixed,3. Mixed,3. Mixed,3815942.0,3016970.0,3562937.0,3743862.0,5313963.0,5925172.0,6081336.0


## Check the data

Manually check that our data looks how we expect for selected years.

In [47]:
# Check the totals match those in the published ERIC report files.
assert df.mrc_total_202324.sum() == 244655329
assert df.mrc_on_202324.sum() ==  178363050
assert df.mrc_off_202324.sum() == 66292279
assert df.mrc_total_202223.sum() == 237702259
assert df.mrc_on_202223.sum() == 170928754
assert df.mrc_off_202223.sum() == 66773505
assert df.mrc_total_201718.sum() == 264338474
assert df.mrc_on_201718.sum() == 214650637
assert df.mrc_off_201718.sum() == 49687837

In [35]:
# Check the totals for at an individual trust.
df_test = df[df.trust_code == 'R0A']
assert df_test['mrc_on_201718'].item() == 3612846	
assert df_test['mrc_off_201718'].item() == 203096
assert df_test['mrc_total_201718'].item() == 3612846+203096
assert df_test['mrc_on_201819'].item() == 2871339	
assert df_test['mrc_off_201819'].item() == 145631
assert df_test['mrc_total_201819'].item() == 2871339+145631
assert df_test['mrc_on_201920'].item() == 3285737	
assert df_test['mrc_off_201920'].item() == 277200
assert df_test['mrc_total_201920'].item() == 3285737+277200
assert df_test['mrc_on_202021'].item() == 3285787	
assert df_test['mrc_off_202021'].item() == 458075
assert df_test['mrc_total_202021'].item() == 3285787+458075
assert df_test['mrc_on_202122'].item() == 4875116	
assert df_test['mrc_off_202122'].item() == 438847
assert df_test['mrc_total_202122'].item() == 4875116+438847
assert df_test['mrc_on_202223'].item() == 5629069
assert df_test['mrc_off_202223'].item() == 296103
assert df_test['mrc_total_202223'].item() == 296103+5629069
assert df_test['mrc_on_202223'].item() == 5629069
assert df_test['mrc_off_202223'].item() == 296103
assert df_test['mrc_total_202223'].item() == 296103+5629069
assert df_test['mrc_on_202324'].item() == 5557232	
assert df_test['mrc_off_202324'].item() == 524104
assert df_test['mrc_total_202324'].item() == 5557232+524104

## Output to CSV

Our data is now ready to analyse: output a clean CSV, that we can explore in other notebooks.

In [48]:
df.to_csv('./data/output/trust_mrc_clean.csv', index=False)